In [ ]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.stattools import adfuller

# Cargar los datos (ajustar la ruta a tus archivos)
venta_df_diaria = pd.read_excel("../data/Venta.xlsx")  # Ajusta la ruta según corresponda

# Convertir la columna 'fecha' a tipo datetime
venta_df_diaria['fecha'] = pd.to_datetime(venta_df_diaria['fecha'], format='%d/%m/%Y')

# Agrupar por día y calcular la suma de 'monto_recibido'
daily_sales = venta_df_diaria.groupby('fecha')['monto_recibido'].sum().reset_index()

# Asegurarse de que 'daily_sales' tiene frecuencia diaria y rellenar fechas faltantes con 0
daily_sales = daily_sales.set_index('fecha').asfreq('D', fill_value=0).reset_index()

# Dividir los datos en 70% entrenamiento, 15% validación y 15% prueba
n = len(daily_sales)
train_size = int(n * 0.7)
val_size = int(n * 0.15)

train = daily_sales[:train_size]
val = daily_sales[train_size:train_size + val_size]
test = daily_sales[train_size + val_size:]

# Concatenar los conjuntos de entrenamiento y validación
trainval = pd.concat([train, val])

# Verificar las divisiones de los datos
print(f"Train: {len(train)} registros")
print(f"Validation: {len(val)} registros")
print(f"Test: {len(test)} registros")

# Aplicar la prueba de Dickey-Fuller a la columna 'monto_recibido' para verificar estacionariedad
resultado_adf = adfuller(train['monto_recibido'])
print(f'ADF Statistic: {resultado_adf[0]}')
print(f'p-value: {resultado_adf[1]}')

if resultado_adf[1] > 0.05:
    print("No es estacionaria. Aplicar diferenciación.")
else:
    print("Serie estacionaria. Se puede aplicar ARIMA directamente.")

# Seleccionar el mejor modelo ARIMA utilizando el AIC (sin diferencia si ya es estacionario)
parametros = [(1,0,1), (2,0,2), (2,1,1), (3,0,1)]
mejores_resultados_aic = []

for orden in parametros:
    modelo = ARIMA(train['monto_recibido'], order=orden)
    modelo_fit = modelo.fit()
    aic = modelo_fit.aic
    mejores_resultados_aic.append((orden, aic))
    print(f"ARIMA{orden} → AIC: {aic:.2f}")

# Seleccionar el mejor modelo según AIC
mejor_modelo_aic = sorted(mejores_resultados_aic, key=lambda x: x[1])[0]
print(f"\nMejor modelo según AIC: ARIMA{mejor_modelo_aic[0]} con AIC: {mejor_modelo_aic[1]:.2f}")

# Ajustar el modelo ARIMA con el conjunto de entrenamiento + validación (trainval)
modelo_final = ARIMA(trainval['monto_recibido'], order=mejor_modelo_aic[0])
modelo_final_fit = modelo_final.fit()

# Predicción para los próximos 15 días en el conjunto de prueba
pred_test = modelo_final_fit.forecast(steps=len(test))

# Evaluación del modelo con métricas
mae = mean_absolute_error(test['monto_recibido'], pred_test)
rmse = np.sqrt(mean_squared_error(test['monto_recibido'], pred_test))
smape = 100 * np.mean(2 * np.abs(pred_test - test['monto_recibido']) / (np.abs(test['monto_recibido']) + np.abs(pred_test)))

# Mostrar las métricas de evaluación
metricas = pd.DataFrame({
    'Métrica': ['MAE', 'RMSE', 'sMAPE'],
    'Valor': [round(mae, 2), round(rmse, 2), f"{smape:.2f}%"]
})

print(metricas)

# Graficar los resultados de ARIMA y la diferencia con las ventas históricas
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(train['fecha'], train['monto_recibido'], label='Train')
ax.plot(val['fecha'], val['monto_recibido'], label='Validation')
ax.plot(test['fecha'], test['monto_recibido'], label='Test')
ax.plot(test['fecha'], pred_test, label='Predicción ARIMA', linestyle='--', color='black')
ax.set_title("Predicción de ventas ARIMA")
ax.set_xlabel("Fecha")
ax.set_ylabel("Total Pagado")
ax.legend()
ax.grid(True)
for pos in ['top', 'right', 'left', 'bottom']:
    ax.spines[pos].set_visible(False)
plt.tight_layout()
plt.show()

# Distribución del Error Absoluto
error_absoluto = abs(test['monto_recibido'] - pred_test)
plt.figure(figsize=(10, 4))
plt.hist(error_absoluto, bins=20, color='royalblue', edgecolor='black')
plt.title("Distribución del Error Absoluto en Predicción", fontsize=14)
plt.xlabel("Error Absoluto")
plt.ylabel("Frecuencia")
plt.tight_layout()
plt.show()
